# Visualize Calibration Process

In [ ]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization,utils,impedance_functions
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs, route_utils

In [ ]:
#bring in a calibration result
with (config['calibration_fp'] / 'results/bootsample_994,validation,0.pkl').open('rb') as fh:
    calibration_results = pickle.load(fh)

In [ ]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)
with (config['calibration_fp']/'ready_for_calibration_stats.pkl').open('rb') as fh: # has loss values for shortest path
    full_set = pickle.load(fh)
full_ods = utils.match_results_to_ods_w_year(full_set)
# for adding coordinates to the viz part
with (config['cycleatl_fp']/"rdp.pkl").open('rb') as fh:
    coords = pickle.load(fh)

with (config['cycleatl_fp']/"trips_2.pkl").open('rb') as fh:
    trips = pickle.load(fh)
with (config['cycleatl_fp']/"users_2.pkl").open('rb') as fh:
    users = pickle.load(fh)
trips.set_index('tripid',inplace=True)
users.set_index('userid',inplace=True)
trips = trips.loc[list(full_set.keys())]
users = users.loc[users.index.isin(set(list(trips['userid'])))]
user_map = trips['userid'].to_dict()

In [ ]:
# these are the betas we need
list_of_betas = calibration_results['results']['xall']
funall = calibration_results['results']['funall']
betas = [x['beta'] for x in calibration_results['betas_tup']]
betas_tup = calibration_results['betas_tup']
set_to_zero = calibration_results['set_to_zero']
set_to_inf = calibration_results['set_to_inf']
tripid = 9272

o = full_set[tripid]['origin_node']
d = full_set[tripid]['destination_node']
year = full_set[tripid]['trip_start_time']

# accounts for the trip date
# if infra is on street (i.e., the link is still traversable but the impedance doesn't apply)
links.loc[links['year'] > year,set_to_zero] = 0 
# if it's off-street then assign it a very high cost
links.loc[(links['year'] > year) & (links.loc[:,set_to_inf]==1).any(axis=1),'link_cost_override'] = True

In [ ]:
# [ full_set[tripid]['matched_edges'].values
chosen = full_set[tripid]['matched_edges'].values
from shapely.ops import LineString
chosen = route_utils.get_route_line(chosen,geo_dict)
start_pt = gpd.GeoSeries(Point(chosen[0]),crs=config['projected_crs_epsg'])
end_pt = gpd.GeoSeries(Point(chosen[-1]),crs=config['projected_crs_epsg'])
chosen = gpd.GeoSeries(LineString(chosen),crs=config['projected_crs_epsg'])

In [ ]:
turn_G_copy = turn_G.copy()        

# run network update
rustworkx_routing_funcs.impedance_update(betas,betas_tup,
        impedance_functions.link_impedance_function,
        'travel_time_min',
        'test',
        impedance_functions.turn_impedance_function,
        links,turns,turn_G_copy)

# re-add virtual links
added_nodes = rustworkx_routing_funcs.add_virtual_edges([o],[d],links,turns,turn_G_copy)
_, shortest_paths = rustworkx_routing_funcs.rx_shortest_paths((o,d),turn_G_copy)
shortest_paths = shortest_paths[0]
rustworkx_routing_funcs.remove_virtual_links(added_nodes,turn_G_copy)

In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

iterable = zip(funall.flatten(),chunker(list_of_betas.flatten(),11))

In [ ]:

#TODO, don't run shortest paths, form all of the graphs first
graphs = []

for func, betas in tqdm(iterable,total=len(funall.flatten())):
        turn_G_copy = turn_G.copy()        
        # run network update
        rustworkx_routing_funcs.impedance_update(betas,betas_tup,
                impedance_functions.link_impedance_function,
                'travel_time_min',
                'test',
                impedance_functions.turn_impedance_function,
                links,turns,turn_G_copy)
        if (links['link_cost'] < 0).any() | (turns['turn_cost'] < 0).any() | (turns['total_cost'] < 0).any():
                graphs.append(None)
                continue
        # re-add virtual links
        rustworkx_routing_funcs.add_virtual_edges([o],[d],links,turns,turn_G_copy)
        graphs.append(turn_G_copy)

In [ ]:
x = []
for graph in graphs:
    if graph is None:
        x.append(None)
        continue
    _, shortest_paths = rustworkx_routing_funcs.rx_shortest_paths((o,d),graph)
    shortest_paths = LineString(route_utils.get_route_line(shortest_paths[0],geo_dict))
    # rustworkx_routing_funcs.remove_virtual_links(added_nodes,turn_G_copy)
    x.append(shortest_paths) 

In [ ]:
with (config['scratch_fp']/'viz.pkl').open('wb') as fh:
    pickle.dump(x,fh)

In [ ]:
with (config['scratch_fp']/'viz.pkl').open('rb') as fh:
    big_list = pickle.load(fh)

In [ ]:
chosen.coords

In [ ]:
modeled = gpd.GeoSeries(big_list).set_crs(config['projected_crs_epsg'])
tolerance = 2000
xmin, ymin, xmax, ymax = modeled.total_bounds

In [ ]:
import contextily as cx
import xyzservices as xyz
import shapely

stamen_toner = xyz.TileProvider(
    name="Stamen Toner",
    url=stadia_toner['tiles'],
    attribution=stadia_toner['attr']
)


In [ ]:
modeled_chunked = list(chunker(modeled,25))

In [ ]:
tripid

In [ ]:
if (config['figures_fp'] / 'animation').exists() == False:
    (config['figures_fp'] / 'animation').mkdir()

In [ ]:
for idx, chunk in enumerate(modeled_chunked):
    idx = idx + 1
    infeasible = int(np.sum([1 for x in chunk if x is None]))
    fig, ax = plt.subplots()
    ax.set_xlim([xmin-tolerance,xmax+tolerance])
    ax.set_ylim([ymin-tolerance,ymax+tolerance])
    ax.axis("off")    
    chunk.plot(ax=ax,label='Modeled',color='black',zorder=3)
    chosen.plot(ax=ax,color='red',label='Chosen',zorder=9)
    start_pt.plot(ax=ax,label='Start',zorder=10,color='green',markersize=100)
    end_pt.plot(ax=ax,label='End',zorder=10,color='red',markersize=100)
    ax.legend()
    cx.add_basemap(ax,source=stamen_toner,crs=modeled.crs)
    plt.suptitle(f'Trip ID: {tripid}, Iteration: {idx}, Infeasible: {infeasible}')
    plt.savefig(config['figures_fp']/f'animation/{idx}.png')

In [ ]:
import imageio
images = []

image_fps = sorted(list((config['scratch_fp']/'animation').glob('*.png')))
for fp in image_fps:
    image = imageio.imread(fp)
    images.append(image)
imageio.mimsave(config['figures_fp']/'animation/animation.gif',images,duration=1)


In [ ]:


with imageio.get_writer(config['scratch_fp']/'animation/animation.gif',mode='I') as writer:
    for filename in list((config['scratch_fp']/'animation').glob('*.png')):
        
        writer.append_data(image)



In [ ]:
# find max bounds to use
chosen = gpd.GeoSeries(chosen).set_crs(config['projected_crs_epsg'])






In [ ]:
print(np.sum([1 for x in big_list[1] if x is None]),'na particles')

# find max bounds to use
modeled = gpd.GeoSeries(big_list[1]).set_crs(config['projected_crs_epsg'])


xmin, ymin, xmax, ymax = modeled.total_bounds

fig, ax = plt.subplots()

tolerance = 2000
ax.set_xlim([xmin-tolerance,xmax+tolerance])
ax.set_ylim([ymin-tolerance,ymax+tolerance])
ax.axis("off")

modeled.plot(ax=ax)
chosen.plot(ax=ax,color='red')

cx.add_basemap(ax,source=stamen_toner,crs=modeled.crs)

In [ ]:
big_list

In [ ]:
pop_size = 5
new = []
for x in range(pop_size):
    new.append(all_shortest_paths[x::pop_size])

In [ ]:
len(betas_tup)

In [ ]:
# for idx, iteration in enumerate(new):
#     for mls, iteration:
#         for line in mls:
#             x, y = line.xy
#             ax.plot(x,y,colors)

In [ ]:
# for calibration_iter in all_shortest_paths:

    

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import imageio
from io import BytesIO

# Function to plot a GeoSeries and save the plot
def plot_geoseries(geoseries,other_geoseries,i,past_val):
    fig, ax = plt.subplots(figsize=(20, 20))
    #cx.add_basemap(ax)
    other_geoseries.plot(ax=ax,color='blue',weight=1)
    geoseries.plot(ax=ax,color='red')
    ax.set_title(f"Iter:{i} Overlap Function:{past_val}")
    ax.set_axis_off()
    img_bytes = BytesIO()
    plt.savefig(img_bytes, format='png', bbox_inches='tight')
    plt.close()
    return img_bytes.getvalue()




In [ ]:
# need to generate routes using the past betas for only one trip
